In [1]:
import pandas as pd

from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

In [2]:
df_nutrients = pd.read_csv("rdi.csv")

In [3]:
df_nutrients

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Fibres (g/d),LA (g/d),ALA (g/d),EPA (g/d),DHA (g/d),Calcium (mg/d),...,Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
0,2500,96.0,281.25,97.222222,30.0,11.111111,1.388889,0.125,0.125,1000.0,...,70.0,110.0,1.0467,1.6,16.7472,5.0,1.7,330.0,4.0,750.0


In [4]:
nutrients = [(key,value) for key,value in df_nutrients.to_dict("records")[0].items()]
nutrients

[('Energy (kcal/d)', 2500),
 ('Protein (g/d)', 96.0),
 ('Carbohydrate (g/d)', 281.25),
 ('Fat (g/d)', 97.22222222222224),
 ('Fibres (g/d)', 30.0),
 ('LA (g/d)', 11.11111111111111),
 ('ALA (g/d)', 1.3888888888888888),
 ('EPA (g/d)', 0.125),
 ('DHA (g/d)', 0.125),
 ('Calcium (mg/d)', 1000.0),
 ('Chloride (mg/d)', 3100.0),
 ('Copper (mg/d)', 1.6),
 ('Iron (mg/d)', 11.0),
 ('Iodine (µg/d)', 150.0),
 ('Magnesium (mg/d)', 350.0),
 ('Manganese (mg/d)', 3.0),
 ('Phosphorus (mg/d)', 550.0),
 ('Potassium (mg/d)', 3500.0),
 ('Selenium (µg/d)', 70.0),
 ('Sodium (mg/d)', 2000.0),
 ('Zinc (mg/d)', 10.0),
 ('Vitamin D (µg/d)', 15.0),
 ('Vitamin E (mg/d)', 13.0),
 ('Vitamin K1 (µg/d)', 70.0),
 ('Vitamin C (mg/d)', 110.0),
 ('Vitamin B1 or Thiamin (mg/d)', 1.0467000000000002),
 ('Vitamin B2 or Riboflavin (mg/d)', 1.6),
 ('Vitamin B3 or Niacin (mg/d)', 16.747200000000003),
 ('Vitamin B5 or Pantothenic acid (mg/d)', 5.0),
 ('Vitamin B6 (mg/d)', 1.7),
 ('Vitamin B9 or Folate (µg/d)', 330.0),
 ('Vitamin B1

In [5]:
df_foods=pd.read_csv("ciqual_2020.csv")

In [6]:
# https://stackoverflow.com/questions/11350770/filter-pandas-dataframe-by-substring-criteria
def filter_rows_by_substrings(df, col, substrings):
    return df[~df[col].str.contains('|'.join(substrings))]


# https://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [7]:
# remove certain foods groups, subgroups and subsubgroups

remove_groups = ["baby food", "sugar and confectionery", "beverages", "fats and oils", "starters and dishes"]

df_foods_f1 = filter_rows_by_values(df_foods, "Group", remove_groups)

remove_subgroups = ["savoury biscuits", "breads and similar", "spices", "sauces", "herbs", "cheese and similar"]

df_foods_f2 = filter_rows_by_values(df_foods_f1, "Subgroup", remove_subgroups)

remove_subsubgroups = []

df_foods_f3 = filter_rows_by_values(df_foods_f2, "Subsubgroup", remove_subsubgroups)

In [8]:
substrings_process = ["crisps", 
                      "powder", 
                      "flour", 
                      "dried", 
                      "dehydrated", 
                      "pastry", "Pastry", 
                      "starch", 
                      "Mix", 
                      "pre-cooked", "precooked", 
                      "parboiled", "bulgur",
                      "pasta", 
                      "Condensed", 
                      "canned", 
                      "preserved", 
                      "bran",
                      "pulp",   # selects very exotic fruits only
                     ]

substrings_foods = ["Rillettes. duck", # bug in data: does it really have that much Vitamin C?
                    "Shortbread", # bug in data: does not have a group
                    "Nutritional yeast", "Soy lecithin", "Poppy. seed", "Sesame seed", # unhealthy/uneatable in the suggested amount
                    "Almond paste or marzipan. prepacked", "Raisin", "Royal jelly" # processed food 
                   ]

substrings = substrings_process + substrings_foods

df_foods_filtered = filter_rows_by_substrings(df_foods_f3, "Name", substrings)

In [9]:
commodities = list(df_foods_filtered["Name"])

In [10]:
data = df_foods_filtered.drop(["Group", "Subgroup", "Subsubgroup", "Name"], axis=1).values.tolist()

In [11]:
solver = pywraplp.Solver.CreateSolver('GLOP')

In [12]:
# Declare an array to hold our variables. 
foods = [solver.NumVar(0.0, solver.infinity(), item) for item in commodities]

print('Number of variables =', solver.NumVariables())

Number of variables = 1408


In [13]:
# Create the constraints, one per nutrient. (data = nutrients_per_100_gramm)
# gurobipy can express a lists or arrays of constraints with a nicer DSL 
# instead of the many loops necessary with OR-Tools
constraints = []
for i, nutrient in enumerate(nutrients):
    constraints.append(solver.Constraint(nutrient[1], solver.infinity(), nutrient[0]))
    for j, item in enumerate(data):
        constraints[i].SetCoefficient(foods[j], item[i])

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 33


In [14]:
# Objective function: Minimize the sum of (price-normalized) foods.
objective = solver.Objective()
for i, food in enumerate(foods):
    objective.SetCoefficient(food, 1.0)
objective.SetMinimization()

In [15]:
status = solver.Solve()

# Check that the problem has an optimal solution.
if status != solver.OPTIMAL:
    print('The problem does not have an optimal solution!')
    if status == solver.FEASIBLE:
        print('A potentially suboptimal solution was found.')
    else:
        print('The solver could not solve the problem.')
        exit(1)

In [16]:
# Display the amounts (in dollars) to purchase of each food.
print('\nDaily Foods:')
for i, food in enumerate(foods):
    if food.solution_value() > 0.0:
        print('{}: {} gr'.format(commodities[i], food.solution_value()*100))
print('\nOptimal daily weight: {:.4f} gr'.format(objective.Value()*100))


Daily Foods:
Curly kale. raw: 3.2766162486188195 gr
Black currant. raw: 50.71173733500841 gr
Flaxseed. brown: 0.8382613274888209 gr
Cashew nut. dry-grilled. unsalted: 108.64288816205443 gr
Soybean. whole grain: 42.48022140071488 gr
Amaranth. raw: 211.53842794164106 gr
Sorghum. whole. raw: 13.239419678411293 gr
Buckwheat. whole. raw: 167.99129389021164 gr
Cod liver. raw: 31.45251113981541 gr
Shrimp. cooked: 45.18394382855675 gr
Pure sea salt. no enrichment: 4.705397229139895 gr

Optimal daily weight: 680.0607 gr


In [17]:
food_per_day=[]

nutrients_result = [0] * len(nutrients)
for i, food in enumerate(foods):
    if food.solution_value() > 0.0:
        # default value for weight optimization
        nutrient_per_goal = 100  
        food_per_day.append((commodities[i], food.solution_value() * nutrient_per_goal))
        for j, _ in enumerate(nutrients):
            nutrients_result[j] += data[i][j] * food.solution_value()

df_food_per_day = pd.DataFrame(food_per_day, columns=("food", "weight"))
df_food_per_day.sort_values(by='weight', ascending=False)

,food,weight
5,Amaranth. raw,211.538428
7,Buckwheat. whole. raw,167.991294
3,Cashew nut. dry-grilled. unsalted,108.642888
1,Black currant. raw,50.711737
9,Shrimp. cooked,45.183944
4,Soybean. whole grain,42.480221
8,Cod liver. raw,31.452511
6,Sorghum. whole. raw,13.239420
10,Pure sea salt. no enrichment,4.705397
0,Curly kale. raw,3.276616


In [18]:
print('\nNutrients per day:')
for i, nutrient in enumerate(nutrients):
    print('{}: {:.2f} (min {})'.format(nutrient[0], nutrients_result[i],
                                       nutrient[1]))


Nutrients per day:
Energy (kcal/d): 2500.00 (min 2500)
Protein (g/d): 96.00 (min 96.0)
Carbohydrate (g/d): 281.25 (min 281.25)
Fat (g/d): 104.98 (min 97.22222222222224)
Fibres (g/d): 41.86 (min 30.0)
LA (g/d): 21.56 (min 11.11111111111111)
ALA (g/d): 1.39 (min 1.3888888888888888)
EPA (g/d): 2.14 (min 0.125)
DHA (g/d): 1.58 (min 0.125)
Calcium (mg/d): 1000.00 (min 1000.0)
Chloride (mg/d): 3100.00 (min 3100.0)
Copper (mg/d): 7.06 (min 1.6)
Iron (mg/d): 35.41 (min 11.0)
Iodine (µg/d): 280.17 (min 150.0)
Magnesium (mg/d): 1407.89 (min 350.0)
Manganese (mg/d): 12.60 (min 3.0)
Phosphorus (mg/d): 2841.91 (min 550.0)
Potassium (mg/d): 3648.95 (min 3500.0)
Selenium (µg/d): 70.00 (min 70.0)
Sodium (mg/d): 2189.29 (min 2000.0)
Zinc (mg/d): 19.56 (min 10.0)
Vitamin D (µg/d): 31.84 (min 15.0)
Vitamin E (mg/d): 13.00 (min 13.0)
Vitamin K1 (µg/d): 70.00 (min 70.0)
Vitamin C (mg/d): 110.00 (min 110.0)
Vitamin B1 or Thiamin (mg/d): 1.29 (min 1.0467000000000002)
Vitamin B2 or Riboflavin (mg/d): 1.94 (m

In [19]:
pd.set_option('display.max_columns', 500)

nutrient_per_food = {}

for i, food in enumerate(foods):
    if food.solution_value() > 0.0:      
        for j, nutrient in enumerate(nutrients):
            if food.name() in nutrient_per_food:
                nutrient_per_food[food.name()].append(data[i][j] * food.solution_value())
            else:
                nutrient_per_food[food.name()]=[data[i][j] * food.solution_value()]
                
foods_df = pd.DataFrame.from_dict(nutrient_per_food, orient='index', columns=[n[0] for n in nutrients])

for i, nutrient in enumerate(nutrients):
    foods_df[nutrient[0]]=(foods_df[nutrient[0]]/nutrients_result[i]*100).round(2)

display(foods_df)     

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Fibres (g/d),LA (g/d),ALA (g/d),EPA (g/d),DHA (g/d),Calcium (mg/d),Chloride (mg/d),Copper (mg/d),Iron (mg/d),Iodine (µg/d),Magnesium (mg/d),Manganese (mg/d),Phosphorus (mg/d),Potassium (mg/d),Selenium (µg/d),Sodium (mg/d),Zinc (mg/d),Vitamin D (µg/d),Vitamin E (mg/d),Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
Curly kale. raw,0.00,0.15,0.05,0.03,0.38,0.02,0.94,0.00,0.00,0.61,0.00,0.37,0.16,0.02,0.08,0.15,0.10,0.34,0.10,0.05,0.10,0.00,0.87,22.33,4.32,0.33,0.36,0.37,0.25,0.47,0.65,0.00,1.95
Black currant. raw,0.00,0.70,1.75,0.42,7.03,0.71,5.48,0.00,0.00,2.90,0.00,0.67,1.68,0.27,0.83,1.13,0.95,4.59,0.80,0.06,0.73,0.00,8.19,0.00,83.44,1.49,0.99,0.91,2.77,1.72,0.81,0.00,0.51
Flaxseed. brown,0.00,0.16,0.01,0.34,0.54,0.24,12.67,0.00,0.01,0.18,0.03,0.15,0.13,0.06,0.18,0.13,0.17,0.17,0.24,0.01,0.22,0.01,0.03,0.05,0.00,0.30,0.00,0.01,0.09,0.07,0.12,0.00,0.00
Cashew nut. dry-grilled. unsalted,26.81,19.69,9.08,50.71,14.79,44.94,7.04,0.51,0.69,4.13,0.84,41.57,17.49,0.00,21.61,15.52,20.26,20.25,31.04,0.40,31.10,0.85,4.85,48.58,0.49,30.27,2.75,3.63,14.54,8.58,5.44,0.00,0.10
Soybean. whole grain,7.12,15.27,3.14,7.77,13.19,17.81,39.46,0.00,0.00,9.35,0.00,5.30,18.83,0.09,7.63,7.62,8.76,20.26,5.34,0.06,6.41,0.00,2.78,28.52,2.32,28.61,19.07,4.11,7.89,7.90,27.26,0.00,0.06
Amaranth. raw,30.88,29.75,44.08,14.15,33.86,26.88,6.24,0.00,0.00,33.63,0.00,15.89,45.46,0.00,37.26,55.92,41.46,29.45,0.00,0.39,31.04,0.00,19.36,0.00,8.08,19.65,21.83,11.62,42.16,58.00,33.93,0.00,0.02
Sorghum. whole. raw,1.85,1.46,3.08,0.44,2.12,0.27,0.51,0.00,0.00,0.17,0.26,0.53,1.26,0.00,1.55,1.69,1.35,1.32,8.13,0.01,1.13,0.00,0.51,0.00,0.00,3.38,0.66,2.92,0.67,2.71,0.52,0.00,0.00
Buckwheat. whole. raw,23.92,22.57,38.59,5.68,28.09,7.48,9.43,0.00,0.00,2.74,0.00,26.19,10.44,1.50,27.56,17.34,21.40,21.18,0.00,0.08,20.61,0.00,0.00,0.00,0.00,13.00,37.27,70.42,28.21,16.39,9.86,0.00,0.00
Cod liver. raw,7.79,1.64,0.00,19.95,0.00,1.63,17.89,94.41,94.46,0.31,0.00,2.94,3.55,56.13,0.18,0.00,1.11,1.12,28.53,8.46,3.12,98.79,48.39,0.00,1.14,2.43,10.55,4.70,2.75,2.19,18.46,35.59,96.78
Shrimp. cooked,1.62,8.61,0.24,0.52,0.00,0.02,0.33,5.08,4.85,45.18,9.78,6.40,1.01,41.93,1.70,0.50,4.45,1.20,25.82,9.45,5.54,0.35,15.01,0.52,0.21,0.52,6.53,1.32,0.68,1.97,2.96,64.41,0.57


In [20]:
foods_df.sum()

Energy (kcal/d)                           99.99
Protein (g/d)                            100.00
Carbohydrate (g/d)                       100.02
Fat (g/d)                                100.01
Fibres (g/d)                             100.00
LA (g/d)                                 100.00
ALA (g/d)                                 99.99
EPA (g/d)                                100.00
DHA (g/d)                                100.01
Calcium (mg/d)                           100.00
Chloride (mg/d)                          100.01
Copper (mg/d)                            100.01
Iron (mg/d)                              100.01
Iodine (µg/d)                            100.00
Magnesium (mg/d)                         100.00
Manganese (mg/d)                         100.00
Phosphorus (mg/d)                        100.01
Potassium (mg/d)                         100.01
Selenium (µg/d)                          100.00
Sodium (mg/d)                            100.00
Zinc (mg/d)                             

In [21]:
import re
def remove_unit(column_name):
    return re.sub(r"\((.*)\)", "", column_name).strip()

In [22]:
for name in nutrient_per_food.keys():
    print(foods_df.loc[[name]].sort_values(name, axis=1, ascending=False).iloc[:, :5].to_dict("index"))

{'Curly kale. raw': {'Vitamin K1 (µg/d)': 22.33, 'Vitamin C (mg/d)': 4.32, 'Vitamin A (µg/d)': 1.95, 'ALA (g/d)': 0.94, 'Vitamin E (mg/d)': 0.87}}
{'Black currant. raw': {'Vitamin C (mg/d)': 83.44, 'Vitamin E (mg/d)': 8.19, 'Fibres (g/d)': 7.03, 'ALA (g/d)': 5.48, 'Potassium (mg/d)': 4.59}}
{'Flaxseed. brown': {'ALA (g/d)': 12.67, 'Fibres (g/d)': 0.54, 'Fat (g/d)': 0.34, 'Vitamin B1 or Thiamin (mg/d)': 0.3, 'LA (g/d)': 0.24}}
{'Cashew nut. dry-grilled. unsalted': {'Fat (g/d)': 50.71, 'Vitamin K1 (µg/d)': 48.58, 'LA (g/d)': 44.94, 'Copper (mg/d)': 41.57, 'Zinc (mg/d)': 31.1}}
{'Soybean. whole grain': {'ALA (g/d)': 39.46, 'Vitamin B1 or Thiamin (mg/d)': 28.61, 'Vitamin K1 (µg/d)': 28.52, 'Vitamin B9 or Folate (µg/d)': 27.26, 'Potassium (mg/d)': 20.26}}
{'Amaranth. raw': {'Vitamin B6 (mg/d)': 58.0, 'Manganese (mg/d)': 55.92, 'Iron (mg/d)': 45.46, 'Carbohydrate (g/d)': 44.08, 'Vitamin B5 or Pantothenic acid (mg/d)': 42.16}}
{'Sorghum. whole. raw': {'Selenium (µg/d)': 8.13, 'Vitamin B1 or T

In [23]:
pd.set_option('display.max_columns', 500)

nutrient_per_food = {}

for i, food in enumerate(foods):
    if food.solution_value() > 0.0:      
        for j, nutrient in enumerate(nutrients):
            if food in nutrient_per_food:
                nutrient_per_food[food].append(data[i][j] * food.solution_value())
            else:
                nutrient_per_food[food]=[data[i][j] * food.solution_value()]
                
foods_df = pd.DataFrame.from_dict(nutrient_per_food, orient='index', columns=[n[0] for n in nutrients])

for i, nutrient in enumerate(nutrients):
    foods_df[nutrient[0]]=(foods_df[nutrient[0]]).round(2)

# foods_df.loc['total'] = foods_df.iloc[1:, :-1].sum()    
    
display(foods_df)  

,Energy (kcal/d),Protein (g/d),Carbohydrate (g/d),Fat (g/d),Fibres (g/d),LA (g/d),ALA (g/d),EPA (g/d),DHA (g/d),Calcium (mg/d),Chloride (mg/d),Copper (mg/d),Iron (mg/d),Iodine (µg/d),Magnesium (mg/d),Manganese (mg/d),Phosphorus (mg/d),Potassium (mg/d),Selenium (µg/d),Sodium (mg/d),Zinc (mg/d),Vitamin D (µg/d),Vitamin E (mg/d),Vitamin K1 (µg/d),Vitamin C (mg/d),Vitamin B1 or Thiamin (mg/d),Vitamin B2 or Riboflavin (mg/d),Vitamin B3 or Niacin (mg/d),Vitamin B5 or Pantothenic acid (mg/d),Vitamin B6 (mg/d),Vitamin B9 or Folate (µg/d),Vitamin B12 (µg/d),Vitamin A (µg/d)
Curly kale. raw,0.00,0.14,0.14,0.04,0.16,0.00,0.01,0.00,0.00,6.06,0.00,0.03,0.06,0.05,1.10,0.02,2.72,12.39,0.07,1.20,0.02,0.00,0.11,15.63,4.75,0.00,0.01,0.06,0.02,0.01,3.31,0.00,32.38
Black currant. raw,0.00,0.67,4.91,0.44,2.94,0.15,0.08,0.00,0.00,28.96,0.00,0.05,0.59,0.76,11.66,0.14,27.13,167.35,0.56,1.27,0.14,0.00,1.06,0.00,91.79,0.02,0.02,0.15,0.20,0.04,4.16,0.00,8.45
Flaxseed. brown,0.00,0.15,0.03,0.35,0.23,0.05,0.18,0.00,0.00,1.76,0.84,0.01,0.05,0.17,2.51,0.02,4.69,6.29,0.17,0.32,0.04,0.00,0.00,0.04,0.00,0.00,0.00,0.00,0.01,0.00,0.60,0.00,0.01
Cashew nut. dry-grilled. unsalted,670.33,18.90,25.53,53.24,6.19,9.69,0.10,0.01,0.01,41.28,25.97,2.93,6.19,0.00,304.20,1.96,575.81,738.77,21.73,8.69,6.08,0.27,0.63,34.01,0.54,0.39,0.05,0.61,1.06,0.18,27.81,0.00,1.68
Soybean. whole grain,177.99,14.66,8.84,8.16,5.52,3.84,0.55,0.00,0.00,93.46,0.00,0.37,6.67,0.25,107.47,0.96,248.93,739.16,3.74,1.27,1.25,0.00,0.36,19.97,2.55,0.37,0.37,0.69,0.58,0.17,139.34,0.00,0.92
Amaranth. raw,772.12,28.56,123.96,14.85,14.17,5.80,0.09,0.00,0.00,336.35,0.00,1.12,16.10,0.00,524.62,7.04,1178.27,1074.62,0.00,8.46,6.07,0.00,2.52,0.00,8.88,0.25,0.42,1.95,3.09,1.25,173.46,0.00,0.35
Sorghum. whole. raw,46.21,1.40,8.66,0.46,0.89,0.06,0.01,0.00,0.00,1.72,7.94,0.04,0.44,0.00,21.85,0.21,38.26,48.06,5.69,0.26,0.22,0.00,0.07,0.00,0.00,0.04,0.01,0.49,0.05,0.06,2.65,0.00,0.00
Buckwheat. whole. raw,598.05,21.67,108.52,5.96,11.76,1.61,0.13,0.00,0.00,27.38,0.00,1.85,3.70,4.20,388.06,2.18,608.13,772.76,0.00,1.68,4.03,0.00,0.00,0.00,0.00,0.17,0.72,11.79,2.07,0.35,50.40,0.00,0.00
Cod liver. raw,194.69,1.57,0.00,20.95,0.00,0.35,0.25,2.02,1.50,3.15,0.00,0.21,1.26,157.26,2.52,0.00,31.45,40.89,19.97,185.26,0.61,31.45,6.29,0.00,1.26,0.03,0.20,0.79,0.20,0.05,94.36,3.15,1604.08
Shrimp. cooked,40.62,8.27,0.66,0.54,0.00,0.00,0.00,0.11,0.08,451.84,303.18,0.45,0.36,117.48,23.95,0.06,126.52,43.83,18.07,206.94,1.08,0.11,1.95,0.36,0.23,0.01,0.13,0.22,0.05,0.04,15.14,5.69,9.49


In [24]:
foods_df.sum()

Energy (kcal/d)                          2500.01
Protein (g/d)                              95.99
Carbohydrate (g/d)                        281.25
Fat (g/d)                                 104.99
Fibres (g/d)                               41.86
LA (g/d)                                   21.55
ALA (g/d)                                   1.40
EPA (g/d)                                   2.14
DHA (g/d)                                   1.59
Calcium (mg/d)                           1000.01
Chloride (mg/d)                          3100.00
Copper (mg/d)                               7.06
Iron (mg/d)                                35.42
Iodine (µg/d)                             280.17
Magnesium (mg/d)                         1407.89
Manganese (mg/d)                           12.59
Phosphorus (mg/d)                        2841.91
Potassium (mg/d)                         3648.97
Selenium (µg/d)                            70.00
Sodium (mg/d)                            2189.28
Zinc (mg/d)         

In [25]:
activities = solver.ComputeConstraintActivities()
o = [{'Name':c.name(), 'shadow price':c.dual_value(), 'slack (%)': (activities[i] - c.lb())/c.lb()*100} for i, c in enumerate(solver.constraints())]
print(pd.DataFrame(o).round(2))

                                     Name  shadow price  slack (%)
0                         Energy (kcal/d)          0.00       0.00
1                           Protein (g/d)          0.00       0.00
2                      Carbohydrate (g/d)          0.01       0.00
3                               Fat (g/d)          0.00       7.98
4                            Fibres (g/d)          0.00      39.54
5                                LA (g/d)          0.00      94.07
6                               ALA (g/d)          0.04      -0.00
7                               EPA (g/d)          0.00    1608.40
8                               DHA (g/d)          0.00    1167.92
9                          Calcium (mg/d)          0.00       0.00
10                        Chloride (mg/d)          0.00       0.00
11                          Copper (mg/d)          0.00     341.06
12                            Iron (mg/d)          0.00     221.94
13                          Iodine (µg/d)          0.00      8

In [26]:
activities = solver.ComputeConstraintActivities()
o = [{'Name':c.name(), 'shadow price':c.dual_value(), 'slack': (activities[i] - c.lb())} for i, c in enumerate(solver.constraints())]
df_sensitivity = pd.DataFrame(o)
print(df_sensitivity.round(2))

                                     Name  shadow price    slack
0                         Energy (kcal/d)          0.00     0.00
1                           Protein (g/d)          0.00     0.00
2                      Carbohydrate (g/d)          0.01     0.00
3                               Fat (g/d)          0.00     7.76
4                            Fibres (g/d)          0.00    11.86
5                                LA (g/d)          0.00    10.45
6                               ALA (g/d)          0.04    -0.00
7                               EPA (g/d)          0.00     2.01
8                               DHA (g/d)          0.00     1.46
9                          Calcium (mg/d)          0.00     0.00
10                        Chloride (mg/d)          0.00     0.00
11                          Copper (mg/d)          0.00     5.46
12                            Iron (mg/d)          0.00    24.41
13                          Iodine (µg/d)          0.00   130.17
14                       

The constraints with a slack value of zero are the most critical for the solution. 
It these constraints are changed the solution will also change. 
There are much more critical constraint for the calories optimized diet than for the weight optimized diet. 
The higher the shadow price the more sensitive is the objective function to changes of that constraint.
So the most critical constraints are 

In [27]:
df_sensitivity.sort_values("shadow price", ascending=False).loc[df_sensitivity["slack"].round(2) == 0.0].round(2)

,Name,shadow price,slack
6,ALA (g/d),0.04,-0.0
27,Vitamin B3 or Niacin (mg/d),0.02,-0.0
2,Carbohydrate (g/d),0.01,0.0
24,Vitamin C (mg/d),0.00,0.0
22,Vitamin E (mg/d),0.00,0.0
1,Protein (g/d),0.00,0.0
18,Selenium (µg/d),0.00,-0.0
0,Energy (kcal/d),0.00,0.0
9,Calcium (mg/d),0.00,0.0
23,Vitamin K1 (µg/d),0.00,0.0
